In [1]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
pd.options.plotting.backend = "plotly"
root_dir = r"\\bosch.com\dfsrb\DfsDE\DIV\AE\NE-CT\010_Dept\020_EAI\080_Students\EAI1\2022-Jiang\Code\Plot\table"
data_frame=pd.read_excel(os.path.join(root_dir, "NoSR_C_Pred_FULL.xlsx") ) 
time_frame_select = [0,3600] # all range
time_frame_select = [680, 750]
ftsize =35
fig_size = (20, 15)


In [3]:
# Ground Truth data
time_frame = time_frame_select # set time interval in [0,3000]
std_time_mins =  data_frame['Timestamp [s]'] /60 # stander time in minutes
data_frame_itv =data_frame.loc[(std_time_mins>=time_frame[0]) & (std_time_mins<=time_frame[1])]
timestamp = data_frame_itv['Timestamp [s]']/60
z_ch3 = data_frame_itv["Impedance |Z| [Ohm]"] # Impedance of the pcb
    
#  Predicted data  
pred_at = data_frame_itv['Pred Covered water area [%]']
pred_a50 = data_frame_itv['Pred Cumulated area droplets <50 µm']
pred_a75 = data_frame_itv['Pred Cumulated area droplets 50-75 µm']
pred_a100 = data_frame_itv['Pred Cumulated area droplets 75-100 µm']
pred_awb = data_frame_itv['Pred Area of closed water film [%]']

dpi = 30

In [4]:
title = dict(
    text="Plotly Title",
    y=0.9,
    x=0.5,
    xanchor = "center",
    yanchor = "top"
)


# problem:
## plot problem
1. line vs scatter, need compare x
2. marker size too large y
3. line width too big y
4. background not clear ?
5. should overlap the eletrical data and water data (the hardest) y 
Multiple Y Axes and Plotly Express
6. customize line color and marker shape y 
7. y label x label not change y 
8. font size  y
9. legend size, position, title y
10. unvisible one legend y
11. box color adding y
12. different color for y axises y
13. tight x and y title ???
14. show specific y tick y
15. different legend group x
16. sample center name y
17. margin too large
## data problem
1. using cycle order to plot figure is more logical
2. wired ground truth


Reference:
https://www.tutorialspoint.com/plotly/plotly_format_axis_and_ticks.htm

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# create eletrical data and covered area of water droplet for sample p34nsruc
# use plotly to reproduce mengnan's result again

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
lw = 1
dpi=30

fig.add_traces([
    go.Scatter(x=timestamp, y=data_frame_itv["Impedance |Z| [Ohm]"].apply(np.log10), 
               name="log Impedance |Z|", showlegend=False, 
               mode="lines+markers", marker_line_width=lw,
               marker=dict(symbol="diamond", size=2)),

    go.Scatter(x=timestamp, y= data_frame_itv['Pred Cumulated area droplets <50 µm'], name='<50µm', 
               mode="lines+markers", 
               marker_line_color="green", marker_line_width=lw,
               marker=dict(symbol="diamond", size=2, color = "green")),
    
    go.Scatter(x=timestamp, y= data_frame_itv['Pred Cumulated area droplets 50-75 µm'], name='50-75µm', 
               mode="lines+markers",
               marker_line_color="yellow", marker_line_width=lw,
               marker=dict(symbol="diamond", size=2, color = "yellow")),
    
    go.Scatter(x=timestamp, y= data_frame_itv['Pred Cumulated area droplets 75-100 µm'], name='75-100µm', 
               mode="lines+markers",
               marker_line_color="purple", marker_line_width=lw,
               marker=dict(symbol="diamond", size=2, color = "purple")),
    
    go.Scatter(x=timestamp, y= data_frame_itv['Pred Area of closed water film [%]'], name='Closed water film', 
               mode="lines+markers",
               marker_line_color="orange", marker_line_width=lw, 
               marker=dict(symbol="diamond", size=2, color = "orange")),
    
    go.Scatter(x=timestamp, y= data_frame_itv['Pred Covered water area [%]'], name='Total', 
               mode="lines+markers", #marker = "diamond",
               marker_line_color="black", marker_line_width=lw,
               marker=dict(symbol="diamond", size=2, color = "black"))],
    secondary_ys=[False] + [True] * 5           
               )

# Set x-axis title, change gird line color
fig.update_xaxes(title_text="Time t [min]", 
                 range=[680,750],
                 gridcolor="lightgray",
                 showline=True, linewidth=1, linecolor="black", mirror=True,
                 zeroline=False)

# Set y-axes titles, change gird line color
fig.update_yaxes(title_text="log Impedance |Z| [Ω] @ 10<sup>5</sup> Hz",
                 range=[4.4, 5.8],
                 secondary_y=False)
fig.update_yaxes(title_text="accumulated covered area of water droplets [%]",
                 range=[0,90], 
                 secondary_y=True)
fig.update_yaxes(gridcolor="lightgray",
                 showline=True, linewidth=1, linecolor="black", mirror=True)

# change background, position legend, adjust legend font size
fig.update_layout(
    height=15*dpi, width=25*dpi,
    plot_bgcolor="rgba(255,255,255,255)",
    paper_bgcolor="rgba(255,255,255,255)",
    
    legend=dict(
        title="Area of water droplets",
        yanchor="bottom",
        y=0.05,
        xanchor="left",
        x=0.01,
        font=dict(size=10,color="black")
        ),
    yaxis=dict(
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue"),
        dtick=0.5
        ),
    title=dict(
        text="p34nsruc ground truth",
        font=dict(size=24),
        y=0.85,
        yanchor="top",
        x=0.5,
        xanchor="center"),
)

fig.show()
# fig.write_image("./tmp/p34nsruc_edata+water_gt.png")
# TODO generalize function

## Experiment 01: P34SRC Cycle 02 visualization -- 24012023

### e-data and w-data vizualization

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_ewdata(df, metatitle):
        
        t = df["Time [m]"]
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        lw = 1
        dpi=30
        fig.add_traces([
                go.Scatter(x=t, y=df["Impedance |Z| [Ohm]"].apply(np.log10), 
                        name="log Impedance |Z|", showlegend=False, 
                        mode="lines+markers", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2)),

                go.Scatter(x=t, y= df['Cumulated area droplets <50 µm [%]'], name='<50µm', 
                        mode="lines+markers", 
                        marker_line_color="green", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "green")),
                
                go.Scatter(x=t, y= df['Cumulated area droplets 50-75 µm [%]'], name='50-75µm', 
                        mode="lines+markers",
                        marker_line_color="yellow", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "yellow")),
                
                go.Scatter(x=t, y= df['Cumulated area droplets 75-100 µm [%]'], name='75-100µm', 
                        mode="lines+markers",
                        marker_line_color="purple", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "purple")),
                
                go.Scatter(x=t, y= df['Area of closed water film [%]'], name='Closed water film', 
                        mode="lines+markers",
                        marker_line_color="orange", marker_line_width=lw, 
                        marker=dict(symbol="diamond", size=2, color = "orange")),
                
                go.Scatter(x=t, y= df['Covered water area [%]'], name='Total', 
                        mode="lines+markers", #marker = "diamond",
                        marker_line_color="black", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "black"))],
                secondary_ys=[False] + [True] * 5           
                        )

        # Set x-axis title, change gird line color
        fig.update_xaxes(title_text="Time t [min]", 
                        #  range=[680,750],
                        gridcolor="lightgray",
                        showline=True, linewidth=1, linecolor="black", mirror=True,
                        zeroline=False)

        # Set y-axes titles, change gird line color
        fig.update_yaxes(title_text="log Impedance |Z| [Ω] @ 10<sup>4</sup> Hz",
                        #  range=[4.4, 5.8],
                        secondary_y=False)
        fig.update_yaxes(title_text="accumulated covered area of water droplets [%]",
                        #  range=[0,90], 
                        secondary_y=True)
        fig.update_yaxes(gridcolor="lightgray",
                        showline=True, linewidth=1, linecolor="black", mirror=True)

        # change background, position legend, adjust legend font size
        fig.update_layout(
                height=15*dpi, width=25*dpi,
                plot_bgcolor="rgba(255,255,255,255)",
                paper_bgcolor="rgba(255,255,255,255)",
                
                legend=dict(
                title="Area of water droplets",
                yanchor="bottom",
                y=0.05,
                xanchor="left",
                x=0.01,
                font=dict(size=10,color="black")
                ),
                yaxis=dict(
                titlefont=dict(color="blue"),
                tickfont=dict(color="blue"),
                dtick=0.5
                ),
                title=dict(
                text=metatitle,
                font=dict(size=24),
                y=0.85,
                yanchor="top",
                x=0.5,
                xanchor="center"),
        )
        
        return fig

## p34src cycle 2 ground truth

In [6]:
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\library\tmp\p34src_c02_gt.xlsx")
fig = plot_ewdata(df, metatitle=None)
fig.show()

In [32]:
import numpy as np
edata = np.log10(df["Impedance |Z| [Ohm]"].to_numpy() / 1e4)
adata = df["Covered water area [%]"].to_numpy()
norm_edata = edata/ np.max(edata)
norm_adata = adata / np.max(adata)
R = np.corrcoef(x=norm_edata, y=norm_adata)
R


array([[ 1.        , -0.99213676],
       [-0.99213676,  1.        ]])

### new method for p34src 2 cycle ground truth

In [81]:
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\demo_data\p34src\table\waterdroplet_stats164-322.xlsx")
fig = plot_ewdata(df, metatitle=None)
fig.show()

In [86]:
from config.components import WaterdropletsArea
areas_df = df[WaterdropletsArea.category()]
areas_df = areas_df.loc[areas_df['Covered water area [%]'] > 0]
edata = np.log10(df["Impedance |Z| [Ohm]"].where(df["Covered water area [%]"] > 0).dropna().to_numpy() / 1e4)
norm_edata =  1- edata/ np.max(edata)
adata = areas_df["Area of closed water film [%]"].to_numpy()
norm_adata = adata/ np.max(adata)

norm_adatas = areas_df.apply(lambda x: (x/ x.max()), axis=0)
norm_adatas_np = norm_adatas.to_numpy()
x = eainfo_datas = np.concatenate((norm_edata.reshape(-1,1), norm_adatas_np), axis=1)
R = np.corrcoef(x, rowvar=False)
R[0]

array([ 1.        , -0.37367863, -0.50624116, -0.41254183, -0.30182758,
        0.93230334,  0.9632046 ])

In [90]:
np.corrcoef(norm_edata,norm_adata, rowvar=False)

array([[1.        , 0.93230334],
       [0.93230334, 1.        ]])

In [91]:
import pandas as pd
from utils.data_viz import plot_ewdata
df = pd.read_excel("tmp/p34src_c02_gt.xlsx")
fig = plot_ewdata(df, metatitle="p34src cycle 2 ground truth")
fig.show()

KeyError: 'Cumulated area droplets >100 µm [%]'

possible problem
1. very unstable curve compared to p32src --> algorithm problem? mislabeling? 
2. the condensation observation time is before condensation time inferred from electrical reaction
 - condensation time electrical: 238.5 ~ 239.0
 - condensation observation time: 236.5(mislabel?) 238.0
3. can not observe the difference between water droplet with different size

### water droplet number visualization

In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_waterdp_num(df, metatitle):
        
        t = df["Time [m]"]
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        lw = 1
        dpi=30
        fig.add_traces([
                go.Scatter(x=t, y=df["Impedance |Z| [Ohm]"].apply(np.log10), 
                        name="log Impedance |Z|", showlegend=False, 
                        mode="lines+markers", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2)),

                go.Scatter(x=t, y= df['Number of droplets <50 µm [#]'], name='<50µm', 
                        mode="lines+markers", 
                        marker_line_color="green", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "green")),
                
                go.Scatter(x=t, y= df['Number of droplets 50-75 µm [#]'], name='50-75µm', 
                        mode="lines+markers",
                        marker_line_color="yellow", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "yellow")),
                
                go.Scatter(x=t, y= df['Number of droplets 75-100 µm [#]'], name='75-100µm', 
                        mode="lines+markers",
                        marker_line_color="purple", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "purple")),
                
                go.Scatter(x=t, y= df['Area of closed water film [%]'], name='Closed water film', 
                        mode="lines+markers",
                        marker_line_color="orange", marker_line_width=lw, 
                        marker=dict(symbol="diamond", size=2, color = "orange")),
                
                go.Scatter(x=t, y= df['Total Number of droplets Ntotal [#]'], name='Total', 
                        mode="lines+markers", #marker = "diamond",
                        marker_line_color="black", marker_line_width=lw,
                        marker=dict(symbol="diamond", size=2, color = "black"))],
                secondary_ys=[False] + [True] * 5           
                )


        # Set x-axis title, change gird line color
        fig.update_xaxes(title_text="Time t [min]", 
                        #  range=[680,750],
                        gridcolor="lightgray",
                        showline=True, linewidth=1, linecolor="black", mirror=True,
                        zeroline=False)

        # Set y-axes titles, change gird line color
        fig.update_yaxes(title_text="log Impedance |Z| [Ω] @ 10<sup>4</sup> Hz",
                        #  range=[4.4, 5.8],
                        secondary_y=False)
        fig.update_yaxes(title_text="accumulated covered area of water droplets [%]",
                        #  range=[0,90], 
                        secondary_y=True)
        fig.update_yaxes(gridcolor="lightgray",
                        showline=True, linewidth=1, linecolor="black", mirror=True)

        # change background, position legend, adjust legend font size
        fig.update_layout(
                height=15*dpi, width=25*dpi,
                plot_bgcolor="rgba(255,255,255,255)",
                paper_bgcolor="rgba(255,255,255,255)",
                
                legend=dict(
                title="Area of water droplets",
                yanchor="bottom",
                y=0.05,
                xanchor="left",
                x=0.01,
                font=dict(size=10,color="black")
                ),
                yaxis=dict(
                titlefont=dict(color="blue"),
                tickfont=dict(color="blue"),
                dtick=0.5
                ),
                title=dict(
                text=metatitle,
                font=dict(size=24),
                y=0.85,
                yanchor="top",
                x=0.5,
                xanchor="center"),
        )
        
        return fig

In [5]:
df = pd.read_excel("tmp/p34src_c02_gt.xlsx")
fig = plot_waterdp_num(df, metatitle="p34src cycle 2 droplet number ground truth")
fig.show()

## Experiment 02 P34src with improved label

### ewdata

In [94]:
import pandas as pd
df = pd.read_excel("tmp/p34src_c02_gt.xlsx")

fig = plot_ewdata(df, metatitle="p34src cycle 2 GT")

fig.show()

KeyError: 'Cumulated area droplets >100 µm [%]'

## problem
there are still some mislabel in this term. we should investigate them frame by frame to improve the quality of the label further.
time point, which are lower than the previous:
246.5
248
250.5
253.5
254
260
266.5
268.5
269.5
271
271.5
278.5
286.5
287
288.5
289.5
291
292
also, we should smooth the curve. but how can we smooth it? investigate methods for it and test

In [4]:
from data_viz import plot_waterdp_num

fig = plot_waterdp_num(df, metatitle=" number of water droplets for p34src cycle 2")
fig.show()

# Experiment 03 P32nSRC cycle 10, 2068 - 2366, 01/02/2023


## ewdata

In [9]:
import pandas as pd
# from utils.data_viz import plot_ewdata

df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\library\tmp\p32nsrc_c10_gt.xlsx")

fig = plot_ewdata(df, metatitle=None)

fig.show()

### new method for p32nsrc 10th cycle ground truth

In [10]:
from utils import data_viz
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsrc\table\waterdroplet_stats2068-2366.xlsx")
fig = data_viz.plot_ewdata(df, metatitle=None)
fig.show()


## number of water droplet

In [10]:
from data_viz import plot_waterdp_num

fig = plot_waterdp_num(df, metatitle=" number of water droplets for p32nsrc cycle 10")
fig.show()

# Experiment 04 P32nSRuC cycle ?, 3000 - 3100, 07/02/2023


## ewdata

In [5]:
import pandas as pd
from data_viz import plot_ewdata

df = pd.read_excel("tmp/p32nsruc_3000-3100_gt.xlsx")

fig = plot_ewdata(df, metatitle="p32nsruc 3000-3100 GT")

fig.show()

## number of water droplet

In [6]:
from data_viz import plot_waterdp_num

fig = plot_waterdp_num(df, metatitle=" number of water droplets for p32nsruc 3000 - 3100")
fig.show()

# 28022023 plot results after 2nd improved label

## P32NSRC

In [4]:
from data_viz import plot_waterdp_num
from dataapi.pcbsample import PCBSample
s = PCBSample(sample="p32nsrc", local=False)
gt_table = s.get_gt_table()
fig = plot_waterdp_num(gt_table, metatitle=" number of water droplets for p32nsrc cycle 10")
fig.show()

predict table is only saved in config/pred
image and table will be saved remotely
sample p32nsrc original video is found. 


In [6]:
from dataapi.pcbsample import PCBSample
from utils import data_viz
s = PCBSample(sample="p32nsrc", local=False)
# gt_table = s.get_gt_table()
fig = data_viz.plot_ewdata(gt_table, metatitle="p32nsrc cycle 10 GT")

fig.show()

predict table is only saved in config/pred
image and table will be saved remotely
sample p32nsrc original video is found. 


In [95]:
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsrc\table\waterdroplet_stats1034.0-1183.0.xlsx")
fig = plot_ewdata(df, metatitle=None)
fig.show()

KeyError: 'Impedance |Z| [Ohm]'

## P32NSRUC

In [8]:
from data_viz import plot_waterdp_num
from dataapi.pcbsample import PCBSample
s = PCBSample(sample="p32nsruc", local=False)
gt_table = s.get_gt_table()
fig = plot_waterdp_num(gt_table, metatitle=f"number of water droplets for {s.sample} 3000-3100")
fig.show()

predict table is only saved in config/pred
image and table will be saved remotely
sample p32nsruc original video is found. 


# reproduce nan's test dataset: p32nsruc_nan, from 1324.5 min to 1420

In [1]:
import pandas as pd
tbl = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\table\waterdroplet_stats109-3577.xlsx"
waterdroplet_data = pd.read_excel(tbl)

In [2]:
waterdroplet_data.columns

Index(['Unnamed: 0', 'Time [m]', 'Cumulated area droplets <50 µm [%]',
       'Cumulated area droplets 50-75 µm [%]',
       'Cumulated area droplets 75-100 µm [%]',
       'Cumulated area droplets >100 µm [%]', 'Area of closed water film [%]',
       'Covered water area [%]', 'Number of droplets <50 µm [#]',
       'Number of droplets 50-75 µm [#]', 'Number of droplets 75-100 µm [#]',
       'Number of droplets >100 µm [#]', 'Number of closed water film [#]',
       'Total Number of droplets Ntotal [#]'],
      dtype='object')

In [6]:
waterdroplet_data['Time [m]'].values
time_minutes = waterdroplet_data['Time [m]'].values
tabledir = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\table"

In [13]:
import os
import numpy as np
import cv2
from utils_bp.data_process import find_poles, comb_droplet_pole, expand
from utils.cos_utils import convert_hms2m
imgpth = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\img"
maskpth = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\mask_nan"
color_map = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\color_map_nan"
tabledir = r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\table"
cleaned = False
save_color_droplets = True

if cleaned is True:
    avg_area_pole = 271537 
    pixel_distance = 27 
    number_of_plastic_pix =  192123  
else:
    avg_area_pole = 345027
    pixel_distance = 25
    number_of_plastic_pix =  205388

delta_area_pole = 14000 
pixel_distance = 25 
real_distance =100 
n_s_list = []
n_m_list = []
n_b_list = []
n_l_list = []
n_cwf_list = []
n_t_list = []
area_s_list = []
area_m_list = []
area_b_list = []
area_l_list = []
area_cwf_list = []
area_t_list = []

for t in time_minutes:
    idx = int(2*t)
    img = os.path.join(imgpth, f"{idx:04d}.jpg")
    mask = os.path.join(maskpth, f"{idx:04d}.png")
    if not os.path.exists(mask):
        n_s_list.append(0)
        n_m_list.append(0)
        n_b_list.append(0)
        n_l_list.append(0)
        n_cwf_list.append(0)
        n_t_list.append(0)

        area_s_list.append(0)
        area_m_list.append(0)
        area_b_list.append(0)
        area_l_list.append(0)
        area_cwf_list.append(0)
        area_t_list.append(0)
    else:
        i = cv2.imread(img)
        m = cv2.imread(mask, cv2.IMREAD_GRAYSCALE) 
        poles_img = expand(find_poles(i)) # from the original image to get the binary metal image
        poles_gray_img = cv2.cvtColor(poles_img, cv2.COLOR_BGR2GRAY) # get the water droplets one by one
        # cv2.imwrite("tmp/gs_viz.png", poles_gray_img)
        _, poles_th = cv2.threshold(poles_gray_img, 127, 255, cv2.THRESH_BINARY)
        ret_pole, _, _, _ = cv2.connectedComponentsWithStats(poles_th, connectivity=8) # get to know if the pole image consist of 2 part
        poles_area_one_img = np.sum(poles_th == 255)

        flag = 0
        if flag == 0: # bug
            poles_img_temp = poles_img
            flag = 1

        if (ret_pole -1 != 2) : 
            # if the poles in this picture are connected, ues the previous one
            # if the area of it changed too much, ues the one before previous one
            poles_img = poles_img_temp

        if (poles_area_one_img > avg_area_pole + delta_area_pole) or (poles_area_one_img < avg_area_pole - delta_area_pole):
            poles_img = poles_img_temp

        poles_img_temp = poles_img # poles_img_temp saves the current pole image
        # area_t: total area
        # n_t: total number of droplet
        area_s, area_m, area_b, area_l, area_cwf, area_t, n_s, n_m, n_b, n_l, n_cwf, n_t, color_img = comb_droplet_pole(m, poles_img, 
                                                                                                            pixel_distance = pixel_distance, 
                                                                                                            real_distance =real_distance, 
                                                                                                            number_of_plastic_pix=  number_of_plastic_pix
                                                                                                            ) # combine the metal image and the droplets mask
        overlaped_img = cv2.bitwise_and(i,i, mask=cv2.cvtColor(poles_img, cv2.COLOR_BGR2GRAY))
        
        n_s_list.append(n_s)
        n_m_list.append(n_m)
        n_b_list.append(n_b)
        n_l_list.append(n_l)
        n_cwf_list.append(n_cwf)
        n_t_list.append(n_t)

        area_s_list.append(area_s)
        area_m_list.append(area_m)
        area_b_list.append(area_b)
        area_l_list.append(area_l)
        area_cwf_list.append(area_cwf)
        area_t_list.append(area_t)
        if save_color_droplets:
            cv2.imwrite(os.path.join(color_map, f"{idx:04d}.png"), cv2.hconcat([i, color_img]))

ewdata = pd.DataFrame({ 
                        # time stamp in min
                        "Time [m]": time_minutes,
                        ## droplet
                        'Total Number of droplets Ntotal [#]': n_t_list,
                        'Number of droplets <50 µm [#]': n_s_list,
                        'Number of droplets 50-75 µm [#]': n_m_list,
                        'Number of droplets 75-100 µm [#]': n_b_list,
                        'Number of droplets >100 µm [#]': n_l_list,
                        'Number of closed water film [#]': n_cwf_list,
                        ## area
                        'Covered water area [%]': area_t_list,
                        'Cumulated area droplets <50 µm [%]': area_s_list,
                        'Cumulated area droplets 50-75 µm [%]': area_m_list,
                        'Cumulated area droplets 75-100 µm [%]': area_b_list,
                        'Cumulated area droplets >100 µm [%]': area_l_list,
                        'Area of closed water film [%]': area_cwf_list,
                        })
ewdata.to_excel(os.path.join(tabledir, "nan_waterdroplet_stats109-3577.xlsx"))

# demostrated water droplet areas/ number

## water droplet number range from 1340 - 1420, dataset P32NSRUC, NAN

### new approach

In [29]:
import pandas as pd
import plotly.express as px
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\table\waterdroplet_stats109-3577.xlsx")
df_focus = df[(df['Time [m]'] > 1340) & (df['Time [m]'] <1420)]
figure = px.line(df_focus, x="Time [m]", y="Total Number of droplets Ntotal [#]", title='new approach', markers=True)
figure.show()

In [31]:
from utils.data_viz import plot_ewdata
figure = plot_ewdata(df_focus, metatitle=None)
figure.show()

### nan approach

In [32]:
import pandas as pd
import plotly.express as px
df = pd.read_excel(r"C:\Users\HAU4SI\Desktop\project\demo_data\p32nsruc_nan\table\nan_waterdroplet_stats109-3577.xlsx")
df_focus = df[(df['Time [m]'] > 1340) & (df['Time [m]'] <1420)]
figure = px.line(df_focus, x="Time [m]", y="Total Number of droplets Ntotal [#]", title='new approach', markers=True)
figure.show()

In [33]:
from utils.data_viz import plot_ewdata
figure = plot_ewdata(df_focus, metatitle=None)
figure.show()